In [3]:
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
import requests,os,sys,json,re,zlib,brotli
from bs4 import BeautifulSoup
from time import sleep
from pprint import pprint
import pandas as pd
from time import sleep

In [5]:
from leagues.models import Season,Competition
from base_app.decorators import catch_iteration_errors
from base_app.helpers import fetch_configurations
from base_app.models import ConfigItems

In [10]:
CONFIG = fetch_configurations()

In [11]:
all_comps_sa = Competition.objects.filter(event_data_available=True)

In [12]:
all_comps_sa[0].name_scoresaway

'AFC Champions League Elite'

## Navigate to page, get all values and links in the select box. SIMPLE

In [13]:
def is_recent(entry, threshold=int(CONFIG.get('START_YEAR'))):
    try:
        # Split by '/' or just wrap in a list if single year
        years = [int(y) for y in str(entry).split('/') if y.isdigit()]
        # Keep if any year is >= threshold
        return any(y >= threshold for y in years)
    except ValueError:
        return False

In [29]:
@catch_iteration_errors
def fetch_event_season_urls(index,comp):
    driver = webdriver.Chrome()
    sleep(5)
    driver.get(comp.event_data_url)
    select_box = driver.find_element(By.CSS_SELECTOR, "#season-select")
    links = {season.get_attribute('innerText') : f"https://www.scoresway.com{season.get_property('value')}" for season in select_box.find_elements(By.CSS_SELECTOR, "option")}
    # print(links)
    driver.quit()
    return links

for i,comp in enumerate(all_comps_sa,1) :
    link_dicts = fetch_event_season_urls(i,comp)
    exisiting_seasons = Season.objects.filter(competition = comp).values_list('name',flat=True)
    if comp.name_scoresaway is None :
        print("nahi hua : ",comp.__dict__)
        print()
        continue
    seasons_considered = {yr:value for yr,value in link_dicts.items() if is_recent(yr) and yr not in exisiting_seasons}
    for season_name, event_url in seasons_considered.items():
        Season.objects.create(competition=comp,name=season_name,season_event_url=event_url)
    print(seasons_considered)


{'2025/2026': 'https://www.scoresway.com/en_GB/soccer/afc-champions-league-elite-2025-2026/5bwi65jd7gzy21z4hzncxv3tg/fixtures'}
{'2025/2026': 'https://www.scoresway.com/en_GB/soccer/afc-champions-league-two-2025-2026/b793jwcz37zcxyw5w3w42qz2s/fixtures'}
{}
{'2025/2026': 'https://www.scoresway.com/en_GB/soccer/a-league-men-2025-2026/f2s3v11z1occwbp69h73coqvo/fixtures'}
{}
{'2026/2027': 'https://www.scoresway.com/en_GB/soccer/j1-league-2026-2027/bakisqj15kviwhxxx3wr3rv9w/fixtures'}
{}
{'2025/2026': 'https://www.scoresway.com/en_GB/soccer/saudi-league-2025-2026/6ru2ri6bqeg9l7av4ppdpdx5g/fixtures'}
{'2025/2026': 'https://www.scoresway.com/en_GB/soccer/thai-league-1-2025-2026/697m6fd66efowwnhfyxgujspg/fixtures'}
{'2025/2026': 'https://www.scoresway.com/en_GB/soccer/caf-champions-league-2025-2026/6ndp82t497erhm0wtk8x8qtxw/fixtures'}
{'2025/2026': 'https://www.scoresway.com/en_GB/soccer/caf-confederation-cup-2025-2026/41hzi8yof723pfmndp50e3dp0/fixtures'}
{'2025/2026': 'https://www.scoresway.c

## FOR FOTMOB IT"LL WORK SAME

In [6]:
all_comps_fm = Competition.objects.filter(shot_data_available=True)

In [ ]:
@catch_iteration_errors
def fetch_shot_season_urls(index,comp):
    driver = webdriver.Chrome()
    driver.get(comp.shot_data_url)
    sleep(5)
    body = {}
    for request in driver.requests:
        if request.response and "https://www.fotmob.com/api/data/leagues" in request.url  :
            encoding = request.response.headers.get('Content-Encoding', '')
            raw_body = request.response.body
            if encoding == 'br':
                body = brotli.decompress(raw_body).decode('utf-8')
            elif encoding == 'gzip':
                body = zlib.decompress(raw_body, zlib.MAX_WBITS | 16).decode('utf-8')
            elif encoding == 'deflate':
                body = zlib.decompress(raw_body).decode('utf-8')
            else:
                body = raw_body.decode('utf-8')
    with open("Example.txt","w",encoding="utf-8") as f :
        f.write(str(body))
    exisiting_seasons = Season.objects.filter(competition = comp).exclude(season_shot_url = "").values_list('name',flat=True)
    fm_seasons = json.loads(body).get('allAvailableSeasons')
    all_seasons = [x for x in fm_seasons if is_recent(x) and x not in exisiting_seasons ]
    all_season_urls = {}
    for season in all_seasons:
        if season not in all_season_urls:
            current_obj = Season.objects.get(competition=comp,name=season)
            link = f'{comp.shot_data_url}?season={season.replace(r"/","-").replace(" ","+")}'
            current_obj.season_shot_url = link
            current_obj.save()
    driver.quit()

for i,comp in enumerate(all_comps_fm,1) :
    link_dicts = fetch_shot_season_urls(i,comp)


In [ ]:
len(all_comps_fm)

In [7]:
jap = next(x for x in all_comps_fm if x.name_fotmob=='J. League')
mex = next(x for x in all_comps_fm if x.name_fotmob=='Liga MX')

In [14]:
driver = webdriver.Chrome()
driver.get(mex.shot_data_url)

for request in driver.requests:
    if request.response and "https://www.fotmob.com/api/data/leagues" in request.url:
        encoding = request.response.headers.get('Content-Encoding', '')
        raw_body = request.response.body
        print(encoding)

        if encoding == 'br':
            body = brotli.decompress(raw_body).decode('utf-8')
        elif encoding == 'gzip':
            body = zlib.decompress(raw_body, zlib.MAX_WBITS | 16).decode('utf-8')
        elif encoding == 'deflate':
            body = zlib.decompress(raw_body).decode('utf-8')
        else:
            body = raw_body.decode('utf-8')
        print(body)
driver.quit()
all_seasons = [x for x in json.loads(body).get('allAvailableSeasons') if is_recent(x)]
# all_season_urls = {}

br
{"tabs":["overview","table","fixtures","transfers","seasons"],"allAvailableSeasons":["2025/2026 - Clausura","2025/2026 - Apertura","2024/2025 - Clausura","2024/2025 - Apertura","2023/2024 - Clausura","2023/2024 - Apertura","2022/2023 - Clausura","2022/2023 - Apertura","2021/2022 - Clausura","2021/2022 - Apertura","2020/2021 - Clausura","2020/2021 - Apertura","2019/2020 - Clausura","2019/2020 - Apertura","2018/2019 - Clausura","2018/2019 - Apertura","2017/2018 - Clausura","2017/2018 - Apertura","2016/2017 - Clausura","2016/2017 - Apertura","2015/2016 - Clausura","2015/2016 - Apertura","2014/2015 - Clausura","2014/2015 - Apertura","2013/2014 - Clausura","2013/2014 - Apertura","2012/2013 - Clausura","2012/2013 - Apertura","2011/2012 - Clausura","2011/2012 - Apertura","2010/2011 - Clausura","2010/2011 - Apertura"],"details":{"id":230,"type":"league","name":"Liga MX","selectedSeason":"2025/2026 - Clausura","latestSeason":"2025/2026 - Clausura","shortName":"Liga MX","country":"MEX","gende

In [15]:
[x for x in json.loads(body).get('allAvailableSeasons') if is_recent(x)][:2]

['2025/2026 - Clausura', '2025/2026 - Apertura']

In [16]:
season_links = {}
for x in all_seasons[:2]:
    season_type = x.split('-')[-1].strip()
    if season_type not in season_links :
        season_links[season_type] = []
    season_links[season_type].append((x,f'{mex.shot_data_url}?season={x.replace(r"/","-").replace(" ","+")}'))

In [17]:
pprint(season_links)

{'Apertura': [('2025/2026 - Apertura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2025-2026+-+Apertura')],
 'Clausura': [('2025/2026 - Clausura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2025-2026+-+Clausura')]}


In [22]:
mexico_items = Season.objects.filter(competition__id=27,name__contains='26')

In [23]:
mexico_items[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x16de25c6f90>,
 'id': 594,
 'competition_id': 27,
 'name': '2025/2026',
 'name_fotmob': '2025/2026 - Apertura',
 'season_event_url': 'https://www.scoresway.com/en_GB/soccer/liga-mx-2025-2026/aegyls91smdw9kipjgbsu8tn8/fixtures',
 'season_shot_url': 'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2025-2026+-+Apertura'}

In [25]:
for i,item in enumerate(mexico_items):
    name,link = season_links['Clausura'][i]
    # name,link = season_links['Clausura'][i]
    print(f"""
            competition = {item.competition}
            name = {item.name}
            name_fotmob = {name}
            season_event_url = {item.season_event_url}
            season_shot_url = {link}
            id = {item.id}
            """)
    Season.objects.create(
            competition = item.competition,
            name = item.name,
            name_fotmob = name,
            season_event_url = item.season_event_url,
            season_shot_url = link)

           # item.save())
    # item.name_fotmob = f"{item.name} - Apertura"
    # item.season_shot_url = link
    
    # item.save()
    print(item.__dict__)
    # break


            competition = Liga MX - Mexico
            name = 2025/2026
            name_fotmob = 2025/2026 - Clausura
            season_event_url = https://www.scoresway.com/en_GB/soccer/liga-mx-2025-2026/aegyls91smdw9kipjgbsu8tn8/fixtures
            season_shot_url = https://www.fotmob.com/leagues/230/overview/liga-mx?season=2025-2026+-+Clausura
            id = 594
            
{'_state': <django.db.models.base.ModelState object at 0x0000016DE25C6F90>, 'id': 594, 'competition_id': 27, 'name': '2025/2026', 'name_fotmob': '2025/2026 - Apertura', 'season_event_url': 'https://www.scoresway.com/en_GB/soccer/liga-mx-2025-2026/aegyls91smdw9kipjgbsu8tn8/fixtures', 'season_shot_url': 'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2025-2026+-+Apertura'}
